In [2]:
%run setup.ipynb


In [19]:
acc = phase2_ar1.accessibility
meta = phase2_ar1.df_samples
bob = phase2_ar1.zarr
bob

<module 'zarr' from '/home/eric/Manuscripts/ag1000g-phase2-data-paper/deps/conda/envs/ag1000g-phase2-data-paper/lib/python3.6/site-packages/zarr/__init__.py'>

In [1]:
# Load packages
import h5py
import allel
import numpy as np
import pandas as pd

# Set phase2 path
phase2_path = '/home/elucas/Liverpool/phase2.AR1/'




In [2]:
# Accessibility
acc_fn = phase2_path + 'accessibility/accessibility.h5'
acc = h5py.File(acc_fn)['2R']['is_accessible']

# Sample meta data
meta_fn = phase2_path + 'samples/samples.meta.txt'
meta = pd.read_csv(meta_fn, sep = '\t')

# Variation
variant_fn = phase2_path + 'variation/main/hdf5/pass/ag1000g.phase2.ar1.pass.h5'
variant_data = h5py.File(variant_fn)['2R']

# Sample names
sample_names = [x.decode('utf-8') for x in variant_data['samples'].value]

# Start and end positions of the dsx target region
startpos = 48714638
endpos = 48714659

In [3]:
genotypes = variant_data['calldata']['genotype']
pos = allel.SortedIndex(variant_data['variants']['POS'])
loc = pos.locate_range(startpos, endpos)
dsx_genotypes = genotypes[loc, :, :]
# There is only one SNP in this region:
print(dsx_genotypes.shape)
# Collapse the two alleles into an allele count per sample, and make this a pandas series
dsx_allele_counts = pd.DataFrame(dsx_genotypes[0,:,:]).apply(sum, axis = 1)
# Count the number of alleles per population
dsx_allele_counts_bypop = dsx_allele_counts.groupby(meta.population).apply(sum)
dsx_allele_counts_bypop['Total'] = np.sum(dsx_allele_counts_bypop)
# Get the total number of haplotypes per population
num_hap_bypop = dsx_allele_counts.groupby(meta.population).count() * 2
num_hap_bypop['Total'] = np.sum(num_hap_bypop)
# Get the frequency of the mutant allele by population
dsx_allele_freq_bypop = pd.DataFrame({'Counts' : dsx_allele_counts_bypop, 'Freq' : dsx_allele_counts_bypop / num_hap_bypop})
dsx_allele_freq_bypop

(1, 1142, 2)


,Counts,Freq
population,,
AOcol,42,0.269231
BFcol,0,0.000000
BFgam,0,0.000000
CIcol,0,0.000000
CMgam,8,0.013468
FRgam,0,0.000000
GAgam,8,0.057971
GHcol,0,0.000000
GHgam,0,0.000000
